In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

%matplotlib inline  

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

data

In [3]:
market_ticker = '^GSPC' #'^GSPC' #'btc-usd' #'tsla'
st = 'f'





fma = 12
sma = 26


stime='2010-12-01'
ftime=None
period = '1d'  # '1d'

In [4]:
def get_data_from_ticker(tick, stime, ftime, interval=period):
    ticker = yf.Ticker(tick)

    df = ticker.history(start=stime, end=ftime, interval=interval)
    x = pd.DataFrame(df)
    x.rename(columns={"Close": tick}, inplace=True)
    z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
    return z

df = pd.DataFrame(get_data_from_ticker(market_ticker, stime, ftime))



df.head(3)

,^GSPC
Date,
2010-11-30,1180.550049
2010-12-01,1206.069946
2010-12-02,1221.530029


In [5]:
def add_percent_update(tick):
    """ date stock's or market's update"""
    gf_copy = df.copy(deep=True)
    gf_copy = pd.DataFrame(gf_copy.drop(df.index[0]))
    gf_copy_v = gf_copy[tick].values
    gf_v = df.copy(deep=True)
    gf_vv = gf_v[tick].values

    for i in range(len(gf_copy)):
        gf_vv[i] = (gf_copy_v[i] - gf_vv[i]) / (gf_vv[i] / 100)

    gf_vv = np.insert(gf_vv, 0, 0)
    gf_vv = np.delete(gf_vv, -1)
    return gf_vv

df[f"% update {market_ticker}"] = add_percent_update(market_ticker)
# gf[f"% update {st}"] = add_percent_update(st)

df.head(3)

,^GSPC,% update ^GSPC
Date,,
2010-11-30,1180.550049,0.000000
2010-12-01,1206.069946,2.161696
2010-12-02,1221.530029,1.281856


In [6]:
r = list(df[f'% update {market_ticker}'].values)

for i in range(len(r)):
    if df.iloc[i, 1] > 0:
        r[i] = 'Plus'
    else:
        r[i] = 'Minus'
        
df['Side'] = r  

In [7]:
''' MACD tests '''

r = list(df[f'% update {market_ticker}'].values)
df['Signal'] = r

df['ma_fast'] = df[market_ticker].ewm(span=fma, adjust=False).mean()
df['ma_slow'] = df[market_ticker].ewm(span=sma, adjust=False).mean()
df['macd'] = df['ma_fast'] - df['ma_slow']
df['signal_macd'] = df['macd'].ewm(span=9, adjust=False).mean()
df['masi'] = df['macd'] - df['signal_macd']

df.head(3)

,^GSPC,% update ^GSPC,Side,Signal,ma_fast,ma_slow,macd,signal_macd,masi
Date,,,,,,,,,
2010-11-30,1180.550049,0.000000,Minus,0.000000,1180.550049,1180.550049,0.000000,0.000000,0.000000
2010-12-01,1206.069946,2.161696,Plus,2.161696,1184.476187,1182.440412,2.035775,0.407155,1.628620
2010-12-02,1221.530029,1.281856,Plus,1.281856,1190.176778,1185.335939,4.840839,1.293892,3.546947


macd data

In [8]:
r = list(df[f'% update {market_ticker}'].values)
df['Signal'] = r

a = -1

for i in range(len(r)):
    try:
        
        ''' signal_macd '''

        if i < sma:
            r[i] = None
            
        elif df.iloc[i, 6] < df.iloc[i, 7] and df.iloc[i, 6] < df.iloc[i-1, 6] and df.iloc[i-1, 6] > df.iloc[i-3, 6] and df.iloc[i, 6] < 0 and df.iloc[i, 6] < 7:
            r[i+1] = 'Short'
            

        else:
             r[i+1] = None

        # else:
        #     pass

    except:
        pass

df['Signal'] = r
df.tail(300)

,^GSPC,% update ^GSPC,Side,Signal,ma_fast,ma_slow,macd,signal_macd,masi
Date,,,,,,,,,
2021-10-28,4596.419922,0.982928,Plus,None,4521.958287,4477.365936,44.592351,26.898469,17.693882
2021-10-29,4605.379883,0.194933,Plus,None,4534.792379,4486.848451,47.943928,31.107561,16.836367
2021-11-01,4613.669922,0.180008,Plus,None,4546.927386,4496.242634,50.684752,35.022999,15.661753
2021-11-02,4630.649902,0.368036,Plus,None,4559.807773,4506.198728,53.609045,38.740208,14.868837
2021-11-03,4660.569824,0.646128,Plus,None,4575.309627,4517.633624,57.676003,42.527367,15.148636
2021-11-04,4680.060059,0.418194,Plus,None,4591.425078,4529.665212,61.759866,46.373867,15.385999
2021-11-05,4697.529785,0.373280,Plus,None,4607.748879,4542.099624,65.649255,50.228945,15.420310
2021-11-08,4701.700195,0.088779,Plus,None,4622.202928,4553.921889,68.281039,53.839363,14.441675
2021-11-09,4685.250000,-0.349878,Minus,None,4631.902477,4563.649897,68.252580,56.722007,11.530573


In [9]:
r = list(df[f'% update {market_ticker}'].values)

for i in range(len(r)):
    try:
        if df.iloc[i, 2] == 'Minus' and df.iloc[i, 3] == 'Short':
            r[i] = True
        elif df.iloc[i, 2] == 'Plus' and df.iloc[i, 3] == 'Long':    
            r[i] = True
        elif df.iloc[i, 2] == 'Minus' and df.iloc[i, 3] == 'Long':    
            r[i] = False  
        elif df.iloc[i, 2] == 'Plus' and df.iloc[i, 3] == 'Short':    
            r[i] = False 
        else:
            r[i] = None
    except:
        pass
    
df['Result'] = r
df['Result'].value_counts()
# df.tail(300)
# df.head(300)

False    25
True     20
Name: Result, dtype: int64

sum

In [10]:
r = list(df[f'% update {market_ticker}'].values)

for i in range(len(r)):
    try:
        if df.iloc[i, 3] == 'Short':
            r[i] = (df.iloc[i, 1] * -1)
        elif df.iloc[i, 3] == 'Long':
            r[i] = df.iloc[i, 1]
        else:
            r[i] = 0
    except:
        pass
    
df['Sum'] = r
df['Sum'].sum()
# df.tail(100)

3.013190903416703